## SIMPLEST SUPPLY/DEMAND DEMO

- Base Model: 
   * 2 techs competing for energy source(lamp and heater), for 2 sources  (electricity and gas)
   * fix supply capital cost to only initial factory cost, not changing costs like labour
   * fix freq parameter (energy saving weight) to 1 (check: no freq for supply??)
   * technologies don't die or become obsolete or change
   * full quantity does not change
   * capital cost does not change
   


TO DO: Find out how the functional form (or at least slope as linear) of demand is known. I was under the impression that only the supply function was fully unknown.

Might be advantageous to build an iterator instead of while loop to find equilibrium

In [1]:
# !jupyter nbconvert --to script supply_demand_new.ipynb

In [2]:
%matplotlib inline

import numpy as np
from copy import deepcopy
import matplotlib.pyplot as plt

from helper.utils import * # for functions `process_subtypes`, `filter_fun`, `init_logger`

In [3]:
# Start a log file (from utils module)
init_logger(filename="supply_demand")



### Fake Data Setup:

This data is structured as if it were the result of passing through the full model until the leaf and 

Note: I introduced a rate parameter to indicate the consumption rate of a tech unit

In [4]:
'''
THINK OF the problem of adjusting fuel quantity when there is demand from different 'cells',
and at different levels
'''

data = {"demand_step":[{"kind": "furnace",
                        "priceUnits": "$",
                        "unit": "GJ",
                        "total_demand": 10000,
                        "leaf": True, # might need to change the 'leaf' paradigm (end use? unique key?)
                        "subtypes": {"furnace1": {"fuel": "electricity",
                                                  "demand": None,
                                                  "price": 200,
                                                  "cap_cost": 10},
                                     "furnace2": {"fuel": "electricity",
                                                  "demand": None,
                                                  "price": 200,
                                                  "cap_cost": 15},
                                     "furnace3": {"fuel": "gas",
                                                  "demand": None,
                                                  "price": 150,
                                                  "cap_cost": 30},
                                     "furnace4": {"fuel": "gas",
                                                  "demand": None,
                                                  "price": 150,
                                                  "cap_cost": 50}
                                     } 
                        },
                        {"kind": "hot_water",
#                          "end_use": True, #?
                         "priceUnits": "$",
                         "unit": "GJ",
                         "total_demand": 9000,
                         "leaf": True,
                         "subtypes": {"hot_water1": {"fuel": "electricity",
                                                     "demand": None, 
                                                     "price": 2,
                                                     "cap_cost": 5},
                                      "hot_water2": {"fuel": "electricity",
                                                     "demand": None,
                                                     "price": 2,
                                                     "cap_cost": 2},
                                      "hot_water3": {"fuel": "gas",
                                                     "demand": None,
                                                     "price": 1.5,
                                                     "cap_cost": 15}
                                      }
                        }],

          "supply_step":[{"kind": "electricity",
                          "leaf": True,
                          "supply": 5000,
                          "subtypes": {"wind": {"price": 110,
                                                "cap_cost": 2},
                                       "hydro": {"price": 100,
                                                 "cap_cost": 5},
                                       "rocks": {"price": 80,
                                                 "cap_cost": 10}
                                       } 
                          },
                          {"kind": "gas",
                           "leaf": True,
                           "supply": 3000,
                           "subtypes": {"gas1": {"price": 150,
                                                 "cap_cost": 10},
                                        "gas2": {"price": 120,
                                                 "cap_cost": 0}
                                        }
                          }]
       }   

#### Example for selecting certain values that meet a requirement with the `process_subtypes` function defined in `utils.py`:

Here showing values in all technologies that use electricity.

In [5]:
# by_elec = filter_obs(data, fuel_type = "electricity", val = "fuel", verbose=True)

#### Define classes for Supply and Demand:

In [6]:
class Demand:
    '''
    
    - **kwargs: dict of named arguments in "argname": value pairs to specify used economic indicators (GDP, population, etc)
    '''

    def __init__(self, data, v=10, return_all_info=True, **kwargs):
        
        self.dtechs = data["demand_step"]
        self.dfuels = data["supply_step"]
        
        lccs = self.lcc_over_tech()
        
        if return_all_info:
            tech_demand, tech_mshares = self.quantity_demanded(lccs, v=v, return_all_info=True)
            fuel_demand = self.demand_by_fuel(tech_demand)
            self.fuel_demand, self.tech_demand, self.tech_mshares = fuel_demand, tech_demand, tech_mshares
        else:
            tech_demand = self.quantity_demanded(lccs, v=v, return_all_info=False)
            fuel_demand = self.demand_by_fuel(tech_demand)
            self.fuel_demand = fuel_demand
        
        
    ## Methods for demand side calculations 
    def life_cycle_cost(self, last_price, cap_cost=0):
        lcc = (last_price + cap_cost)
        return lcc
    
    def market_share(self, lcc, techname, v=10):
        # lcc is a dict
        sum_v = sum(lcc[item]**(-1.0*v) for item in lcc)
        ms = (lcc[techname]**(-1.0*v))/sum_v
        return ms
        
    def get_quantity(self, ms, fuel_quantity):
        demand = ms * fuel_quantity
        return demand
    
    def lcc_over_tech(self, keytech = "kind", keysubtech = "subtypes",
                            keyprice = "price", keycapcost = "cap_cost"):
        # TODO: use some kwargs in lcc method in case there are arbitrary numbers of params
        lccs = {}
        # techdict is the entire subtree for the kth competing tech (demand side)
        for techdict in self.dtechs:
            # get the tech name (key) to use as a label for lcc
            tech_k = techdict[keytech]
            lccs[tech_k] = {}
            # fetch all attributes/values for the (sub)tech that demands a fuel
            # by `subtech` I mean e.g.: tech: furnace, subtech: type of furnace
            subtechs = techdict[keysubtech]            
            # loop over `subtechs`, get subtech name (key) and vals (all attributes associated w/ subtech)
            for subtech, vals in subtechs.items():
                # unpack components needed for lcc
                price = vals[keyprice]
                cc = vals[keycapcost]
                lccs[tech_k][subtech] = self.life_cycle_cost(last_price=price, cap_cost=cc) # kwargs here
        return lccs        
                
                
    def quantity_demanded(self, lccs, v=10, return_all_info=True, keytech="kind", 
                                keysubtech="subtypes", keyquant="total_demand"):
        mshares, quants = {}, {}

        # techdict is the entire subtree for the kth competing tech (demand side)            
        for techdict in self.dtechs:
            # fetch all attributes/values for the (sub)tech that demands a fuel
            # by `subtech` I mean e.g.: tech: furnace, subtech: type of furnace
            subtechs = techdict[keysubtech]
            # loop over `subtechs`, get subtech name (key) and vals (all attributes associated w/ subtech)                
                
            for tech, vals in subtechs.items():
                mshares[tech] = self.market_share(lccs[techdict[keytech]], tech, v=v)
                quants[tech] = self.get_quantity(mshares[tech], techdict[keyquant])     
                    
        if return_all_info:
            return quants, mshares
        else:
            return quants
        
    def demand_by_fuel(self, quants, keyfuel="kind", keytechfuel="fuel", keysubtech="subtypes"):
        by_fuel, sum_quants = {}, {}

        # get only numbers for a given fuel, in both techs
        for fkind in self.dfuels:
            cur_fuel = fkind[keyfuel]
#             cond_func = lambda value: value[keytechfuel] == cur_fuel 
#             by_fuel[cur_fuel] = process_subtypes( self.dtechs, filter_func, cond_func )
            by_fuel[cur_fuel] = filter_obs(data, fuel_type = cur_fuel, val = "fuel", verbose=False)
            sum_quants[cur_fuel] = sum(quants[subtype] for subtype in by_fuel[cur_fuel].keys())    
            
        return sum_quants    
                    



In [7]:
class Supply:
    def __init__(self, data, v=10, return_all_info=True, **kwargs):
        
        self.sfuels = data["supply_step"]
        lccs = self.lcc_over_production()
        
        if return_all_info:
            self.tot_price, self.price, self.sup_mshares = self.get_fuelprice(lccs, v)
        else:
            self.tot_price = self.get_fuelprice(lccs, v)            
            
            
    ## Methods for calculating demand side 
    def life_cycle_cost(self, last_price, cap_cost=0):
        lcc = (last_price + cap_cost)
        return lcc
        
    def market_share(self, lcc, prodname, v=10):
        # lcc is a dict
        sum_v = sum(lcc[item]**(-1.0*v) for item in lcc)
        ms = (lcc[prodname]**(-1.0*v))/sum_v
        return ms
        
    def get_price(self, ms, lcc):
        price = ms*lcc       
        return price
                
        
        
    def lcc_over_production(self, keyfuel = "kind", keyprod = "subtypes",
                                  keyprice = "price", keycapcost = "cap_cost"):  
        lccs = {}        
        for fueldict in self.sfuels:
            # loop through each fuel, fueldict is entire subtree below each type of fuel
            fueltype = fueldict[keyfuel]
            lccs[fueltype] = {}
            
            for prod, vals in fueldict[keyprod].items():
                # loop through each production type (say 'hydro' if fuel is 'electricity')
                lccs[fueltype][prod] = self.life_cycle_cost(last_price=vals[keyprice], cap_cost=vals[keycapcost])
        return lccs    
    
    def get_fuelprice(self, lccs, v = 10, keyfuel = "kind", keyprod = "subtypes",
                           keyprice = "price", keycapcost = "cap_cost",
                           return_all_info = True): 
        # distinguish the 2 types of lccs (one for added, one for price)
        lcc_model, mshares, price, tot_price = {}, {}, {}, {}
        
        for fueldict in self.sfuels:
            # loop through each fuel, fueldict is entire subtree below each type of fuel
            fueltype = fueldict[keyfuel]                          
            lcc_model[fueltype], mshares[fueltype], price[fueltype] = {}, {}, {}
            for prod, vals in fueldict[keyprod].items():               
                lcc_model[fueltype][prod] = self.life_cycle_cost(last_price=vals[keyprice], cap_cost=vals[keycapcost])
                mshares[fueltype][prod] = self.market_share(lccs[fueltype], prod, v=v)                
                price[fueltype][prod] = self.get_price(mshares[fueltype][prod], lcc_model[fueltype][prod])    # (total_quants["electricity"])   
            tot_price[fueltype] = sum(price[fueltype][p]/len(fueldict[keyprod].keys()) for p in fueldict[keyprod].keys())
            # TODO!! change average from regular to weighted with lcc
            
        if return_all_info:
            return tot_price, price, mshares 
        else:
            return tot_price
            
            

In [8]:
class SupplyDemand:
    '''
    SUPPLY-DEMAND MODEL:
        - Demand: estimate Quantity
        - Supply: estimate Price
        - Repeat using updated prices    
    '''    
    def __init__(self, data, verbose=True,
                             demandkey="demand_step",
                             supplykey="supply_step",
                             keyfuel="kind"):
        self.data = data   
        self.techs = data[demandkey]
        self.fuels = data[supplykey]
        self.tempdata = deepcopy(data) # check in case data is unsafe to change
        
        self.demand = Demand(data)
        self.supply = Supply(data)
        
        # may want to make keys editable
        self.reskeys = ("quant_by_fuel", "dem_price_by_fuel", "sup_price_by_fuel")
        self.infokeys = ("price_diff", "quantity_per_tech")
        infovals = []
        
        self.results = dict.fromkeys(self.reskeys, infovals)
        self.info = dict.fromkeys(self.infokeys, infovals)
        self.info["fuels"] = [ft[keyfuel] for ft in self.fuels]

        # Not necessary to store here
        self.info["mshares_demand"] = []
        
        self.initialize()

        
    def initialize(self, keyfuel="kind"):
        
        init_dem = self.get_demand()
        # if verbose...
        dem_mshares = init_dem.tech_mshares
        quant_hat = init_dem.tech_demand
        init_price = {ft[keyfuel]: find_price_for_fuel( fuel_type = ft[keyfuel], data = self.techs ) for ft in self.fuels}
        init_sup = self.get_supply()
        price_hat = init_sup.tot_price
        new_res = [init_price, quant_hat, price_hat]
        
        idx = 0
        for name, result in self.results.items():
            self.results[name].append(new_res[idx])
            idx += 1
            
        self.info["mshares_demand"].append(dem_mshares)
        
        
          
    def iterate(self, relative_tol=0.05, max_iter=1, n_iter=0):
        price_diff, price_thres, ge_thres = {}, {}, {}
        
        for f in self.info["fuels"]:
            last_price = self.results["dem_price_by_fuel"][-1][f]    
            cur_price = self.results["sup_price_by_fuel"][-1][f]
            price_diff[f] = np.abs(last_price - cur_price)
            price_thres[f] = cur_price * relative_tol
            ge_thres[f] = price_diff[f] > price_thres[f]

        self.info["price_diff"].append( price_diff )
        
        while any(ge_thres) and n_iter < max_iter:
            for fuel, is_ge in ge_thres.items():
                new_price = self.results["sup_price_by_fuel"][-1][fuel]
                # change price for fuel belonging to all techs in initial `dataset`
                change_price_for_fuel( self.data["demand_step"], fuel, new_price )

            
            new_dem = self.get_demand()
            dem_mshares = new_dem.tech_mshares
            quant_hat = new_dem.fuel_demand
            new_price = {ft["kind"]: find_price_for_fuel( fuel_type = ft["kind"], data = self.techs ) for ft in self.fuels}
            new_sup = self.get_supply()
            
            price_hat = new_sup.tot_price

            self.results["dem_price_by_fuel"].append(new_price)
            self.results["quant_by_fuel"].append(quant_hat)
            self.results["sup_price_by_fuel"].append(price_hat)
            self.info["mshares_demand"].append(dem_mshares)             
                
    
            for f in self.info["fuels"]:
                last_price = self.results["dem_price_by_fuel"][-1][f]    
                cur_price = self.results["sup_price_by_fuel"][-1][f]
                price_diff[f] = np.abs(last_price - cur_price)
                price_thres[f] = cur_price * relative_tol
                ge_thres[f] = price_diff[f] > price_thres[f]
                
            self.info["price_diff"].append( price_diff )
            n_iter += 1

            
    def get_demand(self):
        return self.demand

    def get_supply(self):
        return self.supply
    
 

In [9]:
sd = SupplyDemand(data)


In [10]:
sd.iterate()

In [11]:
# d = Demand
# dir(sd)

In [12]:
sd.results

{'quant_by_fuel': [{'electricity': 200, 'gas': 150},
  {'furnace1': 1235.9643005551777,
   'furnace2': 976.8193971510727,
   'furnace3': 5773.960694230682,
   'furnace4': 2013.2556080630673,
   'hot_water1': 33.28530647595751,
   'hot_water2': 8966.708407246982,
   'hot_water3': 0.006286277059887388},
  {'electricity': 31.42768280241502, 'gas': 61.06622728811647},
  {'electricity': 0.0, 'gas': 0.0},
  {'electricity': 0.0, 'gas': 0.0},
  {'electricity': 11212.77741142919, 'gas': 7787.222588570809},
  {'electricity': 31.42768280241502, 'gas': 61.06622728811647},
  {'electricity': 0.0, 'gas': 0.0}],
 'dem_price_by_fuel': [{'electricity': 200, 'gas': 150},
  {'furnace1': 1235.9643005551777,
   'furnace2': 976.8193971510727,
   'furnace3': 5773.960694230682,
   'furnace4': 2013.2556080630673,
   'hot_water1': 33.28530647595751,
   'hot_water2': 8966.708407246982,
   'hot_water3': 0.006286277059887388},
  {'electricity': 31.42768280241502, 'gas': 61.06622728811647},
  {'electricity': 0.0, 'g

In [16]:
# d.__doc__

In [12]:
sd.info["price_diff"]

[{'electricity': 200, 'gas': 150},
 {'furnace1': 1235.9643005551777,
  'furnace2': 976.8193971510727,
  'furnace3': 5773.960694230682,
  'furnace4': 2013.2556080630673,
  'hot_water1': 33.28530647595751,
  'hot_water2': 8966.708407246982,
  'hot_water3': 0.006286277059887388},
 {'electricity': 31.42768280241502, 'gas': 61.06622728811647},
 {'electricity': 0.0, 'gas': 0.0},
 {'electricity': 0.0, 'gas': 0.0},
 {'electricity': 11212.77741142919, 'gas': 7787.222588570809},
 {'electricity': 31.42768280241502, 'gas': 61.06622728811647},
 {'electricity': 0.0, 'gas': 0.0},
 {'electricity': 0.0, 'gas': 0.0},
 {'electricity': 11212.77741142919, 'gas': 7787.222588570809},
 {'electricity': 31.42768280241502, 'gas': 61.06622728811647},
 {'electricity': 0.0, 'gas': 0.0},
 {'electricity': 0.0, 'gas': 0.0},
 {'electricity': 11212.77741142919, 'gas': 7787.222588570809},
 {'electricity': 31.42768280241502, 'gas': 61.06622728811647},
 {'electricity': 0.0, 'gas': 0.0},
 {'electricity': 0.0, 'gas': 0.0},
 

In [21]:
sd.results

# CHECK QUANT by fuewl (with tech instead below)

{'quant_by_fuel': [{'electricity': 200, 'gas': 150},
  {'furnace1': 1235.9643005551777,
   'furnace2': 976.8193971510727,
   'furnace3': 5773.960694230682,
   'furnace4': 2013.2556080630673,
   'hot_water1': 33.28530647595751,
   'hot_water2': 8966.708407246982,
   'hot_water3': 0.006286277059887388},
  {'electricity': 31.42768280241502, 'gas': 61.06622728811647}],
 'dem_price_by_fuel': [{'electricity': 200, 'gas': 150},
  {'furnace1': 1235.9643005551777,
   'furnace2': 976.8193971510727,
   'furnace3': 5773.960694230682,
   'furnace4': 2013.2556080630673,
   'hot_water1': 33.28530647595751,
   'hot_water2': 8966.708407246982,
   'hot_water3': 0.006286277059887388},
  {'electricity': 31.42768280241502, 'gas': 61.06622728811647}],
 'sup_price_by_fuel': [{'electricity': 200, 'gas': 150},
  {'furnace1': 1235.9643005551777,
   'furnace2': 976.8193971510727,
   'furnace3': 5773.960694230682,
   'furnace4': 2013.2556080630673,
   'hot_water1': 33.28530647595751,
   'hot_water2': 8966.7084072

In [22]:
sd.iterate()

In [21]:
sd.info

{'price_diff': [{'electricity': 0.0, 'gas': 0.0},
  {'electricity': 0.0, 'gas': 0.0},
  {'electricity': 0.0, 'gas': 0.0},
  {'electricity': 0.0, 'gas': 0.0},
  {'electricity': 0.0, 'gas': 0.0},
  {'electricity': 0.0, 'gas': 0.0}],
 'mshares_demand': [{'furnace1': 0.12359643005551776,
   'furnace2': 0.09768193971510727,
   'furnace3': 0.5773960694230682,
   'furnace4': 0.20132556080630673,
   'hot_water1': 0.0036983673862175017,
   'hot_water2': 0.9963009341385536,
   'hot_water3': 6.984752288763765e-07},
  {'furnace1': 0.12359643005551776,
   'furnace2': 0.09768193971510727,
   'furnace3': 0.5773960694230682,
   'furnace4': 0.20132556080630673,
   'hot_water1': 0.0036983673862175017,
   'hot_water2': 0.9963009341385536,
   'hot_water3': 6.984752288763765e-07},
  {'furnace1': 0.12359643005551776,
   'furnace2': 0.09768193971510727,
   'furnace3': 0.5773960694230682,
   'furnace4': 0.20132556080630673,
   'hot_water1': 0.0036983673862175017,
   'hot_water2': 0.9963009341385536,
   'hot_w

In [39]:
sd.info["price_diff"]

[{'electricity': 0.0, 'gas': 0.0},
 {'electricity': 0.0, 'gas': 0.0},
 {'electricity': 0.0, 'gas': 0.0},
 {'electricity': 0.0, 'gas': 0.0},
 {'electricity': 0.0, 'gas': 0.0},
 {'electricity': 0.0, 'gas': 0.0}]

In [17]:
class SupplyDemand:
    def __init__(self, data):
        
        self.data = data   
        self.techs = data["demand_step"]
        self.fuels = data["supply_step"]
            
        self.results = {}
        self.info = {}

        self.results["quant_by_fuel"] = []
        self.results["dem_price_by_fuel"] = []
        self.results["sup_price_by_fuel"] = []
        
        self.info["price_diff"] = []
        self.info["fuels"] = []
        self.info["mshares_demand"] = []
        self.info["lcc_demand"] = []
        self.info["mshares_supply"] = []
        self.info["quantity_per_tech"] = []
        self.info["price_per_production"] = []
        self.info["n_iters"] = None
        
#        self.initialize()
#         self.iterate()

    def initialize(self, to_condition="fuel"):
        init_dem = self.get_demand(data)
        init_price = {ft["kind"]: find_price_for_fuel( fuel_type = ft["kind"], data = self.techs ) for ft in self.fuels}

        dem_mshares = init_dem.mshares
        quant_hat = init_dem.quants
        
#         init_sup = self.get_supply()
#         price_hat = init_sup.tot_price
        
        self.results["dem_price_by_fuel"].append(init_price)
        self.results["quant_by_fuel"].append(quant_hat)
#         self.results["sup_price_by_fuel"].append(price_hat)
        self.info["mshares_demand"].append(dem_mshares)
    

          
    def iterate(self, relative_tol=0.05, max_iter=100, to_condition="fuel"):
        pass        
        
    def get_demand(self):
        return self.demand

    def get_supply(self):
        pass
    

SyntaxError: invalid character in identifier (<ipython-input-17-d326b980e3f8>, line 2)

In [ ]:
# a = Demand(techs)

# print(a.quants)
# print(a.mshares)

In [ ]:
a.mshares["furnace1"] + a.mshares["furnace2"] + a.mshares["furnace3"] + a.mshares["furnace4"]

In [ ]:
a.mshares["hot_water1"] + a.mshares["hot_water2"] + a.mshares["hot_water3"]

In [ ]:
sup

In [ ]:
b = Supply(sup)

In [ ]:
b.mshares

In [ ]:
b.price

In [ ]:
b.tot_price

In [ ]:
b.mshares["wind"] + b.mshares["hydro"] + b.mshares["rocks"]

In [ ]:
b.mshares["gas1"] + b.mshares["gas2"]

In [ ]:
techs

In [ ]:
sd = SupplyDemand(techs, sup)

In [ ]:
sd.results

In [ ]:
sd.info

In [ ]:
# Note: might want to use sqlite, relational database could be better suited for this data

techs = [{"kind": "furnace",
        "quantity": [10000],
        "priceUnits": "$",
        "unit": "GJ",
        "leaf": True,
        "subtypes": {"type": "furnace1",
                     "fuel": "electricity",
                     "demand": 2000,
                     "price": 2000,
                     "cap_cost": 10},
                     {"type": "furnace2",
                      "fuel": "electricity",
                      "demand": 4000,
                      "price": 2000,
                      "cap_cost": 15},
                       {"type": "furnace3",
                        "fuel": "gas",
                        "demand": 3000,
                        "price": 1500,
                        "cap_cost": 30},
                       {"type": "furnace4",
                        "fuel": "gas",
                        "demand": 1000,
                        "price": 1500,
                        "cap_cost": 100}
                     
        },
        {"kind": "hot_water",
         "quantity": [12000],
         "priceUnits": "$",
         "unit": "GJ",
         "leaf": True,
         "subtypes": {"type": "hot_water1",
                      "fuel": "electricity",
                      "demand": 5000,
                      "price": 2000,
                      "cap_cost": 5},
                     {"type": "hot_water2",
                      "fuel": "electricity",
                      "demand": 2000,
                      "price": 2000,
                      "cap_cost": 2},
                     {"type": "hot_water3",
                      "fuel": "gas",
                      "demand": 5000,
                      "price": 1500,
                      "cap_cost": 15}
                    
        }]

sup = [{"kind": "electricity",
        # "quantity": [total_quants["electricity"]],
        "leaf": True,
        "subtypes": {"wind": {"price": 1200,
                              "cap_cost": 100},
                     "hydro": {"price": 1000,
                                  "cap_cost": 300},
                     "rocks": {"price": 800,
                                  "cap_cost": 200}
                     } 
        },
        {"kind": "gas",
         # "quantity": [total_quants["gas"]],
         "leaf": True,
         "subtypes": {"gas1": {"price": 1500,
                               "cap_cost": 100},
                      "gas2": {"price": 1200,
                               "cap_cost": 200}
                      }
        }]




In [ ]:
class SupplyDemand(Demand):
    def __init__(self, techs, sup):
        
        self.techs = techs
        self.sup = sup
        
#         self.results = {}
        self.dem = None
#         self.results["quantity"] = {}
#         self.results["price"] = {}
#         self.results["marketshare_dem"] = {}
#         self.results["marketshare_sup"] = {}
    
#         self.initialize()
#         self.results["quantity"].append(a)
        
    def run_demand(self):    
        self.dem = Demand(self.techs)
#         return dem.quants, dem.mshares
    
#     def run_supply(self):    
#         supply = Supply(self.sup)
#         return supply.price, supply.mshares
    
#     def initialize(self):
#         quant, msd = self.run_demand()
#         price, msm = self.run_supply()
#         self.results["quantity"] = quant

#         return quant, msd, price, msm
        
#     def iterate(self):
#         quant, msd, price, msm = self.initialize()
#         price_diff 
        
#         while  

In [ ]:
a = SupplyDemand(techs, sup)

In [ ]:
b = a.run_demand()

In [ ]:
b

In [ ]:
results = SupplyDemand(techs, sup, tol_percent=5)

In [ ]:
results.plot("electricity")

In [ ]:
results.plot("gas")

In [ ]:
class SupplyDemand:
    '''
    SUPPLY-DEMAND MODEL:
        - Demand: estimate Quantity
        - Supply: estimate Price
        - Repeat using updated prices
    '''

    def __init__(self, data):
        self.fuelUnits = data["fuelUnits"]
        self.priceUnits = data["priceUnits"]
        self.year = data["year"]
        self.fuelFields = data["fuelType"]
        self.fuelTypes = list(data["fuelType"].keys())
        
        self.numFuels = len(data["fuelType"])
        
        self.techType = []
        self.productQ = []
        
        self.demPrice = []
        self.demCapCost = []        
        self.demLCC = []
        self.demMS = []
        self.demQuantity = []
        
        self.supPrice = []
        self.supCapCost = []               
        self.supLCC = []
        self.supMS = []
        self.supQuantity = []        
        
        self.history = {}
    
        

    def _demand(self):
        
        for fuel, tech_list in self.fuelFields.items():
            self.fuel = fuel
            self.numTechs = len(tech_list)
            
            lcc = []
            ms = []
            quant = []
            rate = []
            
            for tech in range(self.numTechs):
                tech_params = tech_list[tech]
                self.techType.append(tech_params["techType"])
                self.productQ.append(tech_params["productQ"][0])
                self.demPrice.append(tech_params["priceDemanded"][0])
                self.demCapCost.append(tech_params["capCostDemand"][0])
                
                # repeated per tech to simplify, normally should be common to competitive tech (ML! check)
                self.vParam = tech_params["vParam"]
                self.freq = tech_params["freq"]
            
        
                # Estimate Quantity Demanded
                # Get LCC from Price and Capital Cost
                rate.append(tech_params["rate"])
                lcc.append(self.getLCC(price=self.demPrice, capCost=self.demCapCost, freq=self.freq, index = -1))
            ms.append(self.getMS(lcc))

            quant.append(self.getQuantity(ms, self.productQ, rate))
                
                
            self.demLCC.append(lcc)
            self.demMS.append(ms)
            self.demQuantity.append(quant[0])

            
    # Estimate Supply Price
    def _supply(self):
        
        for fuel, tech_list in self.fuelFields.items():
            self.fuel = fuel
            self.numTechs = len(tech_list)
            
            lcc = []
            ms = []
            price = []
            
            for tech in range(self.numTechs):
                tech_params = tech_list[tech]
                self.techType.append(tech_params["techType"])
                self.supCapCost.append(tech_params["capCostSupply"][0])

                # repeated per tech to simplify, normally should be common to competitive tech (ML! check)
                self.vParam = tech_params["vParam"]
                self.freq = tech_params["freq"]
                
        
                # Get LCC from Price and Capital Cost
                print(tech_params["priceSupply"])
                lcc.append(self.getLCC(price=tech_params["priceSupply"], capCost=self.supCapCost, freq=self.freq, index = -1))
                
            ms.append(self.getMS(lcc))
            print(self.demQuantity)
            price.append(self.getPrice(ms, quant=self.demQuantity, lcc=lcc))
                
            self.supLCC.append(lcc)
            self.supMS.append(ms)
            self.supPrice.append(price[0])

        
    def getLCC(self, price, capCost, freq, index = -1):
        # index 0 to pull from list, but the more advanced version uses numpy arrays
        cost = price[index] * freq
        capitalCost = capCost[index]
        tech_lcc = cost + capitalCost
            
        return tech_lcc 
            
        
    def getMS(self, lcc, atol=1e-8):

        full_cost_v = np.sum(np.power(lcc, -1.0*self.vParam))
        ms = np.array([np.power(lcc[nt], -1.0*self.vParam)/full_cost_v for nt in range(self.numTechs)])  
        assert np.isclose(np.sum(ms), 1.0, atol=atol)
        return ms        

    def getQuantity(self, ms, quant, rate, index=-1):
        quant = [ms[0][nt] * quant[nt] * rate[nt] for nt in range(self.numTechs)]
        return quant 
    
    def getPrice(self, ms, lcc, quant):
        weightedP = [ms[0][nr] * quant[nr] * lcc[nr] for nr in range(self.numTechs)]

        return weightedP
        
            

        

In [ ]:
a = SupplyDemand(data)

In [ ]:
a._demand()

In [ ]:
a._supply()

In [ ]:
a.demQuantity

In [ ]:
        self.history["fuel"] = []
        self.history["tech"] = []
        self.history["productQ"] = []
        self.history["quantityDemanded"] = []
        self.history["priceDemanded"] = []
        self.history["capCostDemand"] = []
        self.history["priceSupply"] = []
        self.history["vParam"] = []
        self.history["freq"] = []
        
        self.fuel = ""
        self.techType = ""
        self.productQ = 0
        self.quantityDemanded = 0        
        

In [ ]:
class SupplyDemand:
    '''
    SUPPLY-DEMAND MODEL:
        - Demand: estimate Quantity
        - Supply: estimate Price
        - Repeat using updated prices
    '''

    def __init__(self, data):
        self.techNames = data["techNames"]
        self.fuelNames = data["fuelNames"]
        self.numTechs = len(self.techNames)
        self.numFuels = len(self.fuelNames)
        
        self.productQuantity = self.makeArray(data["productQuantity"], reshape=False)
        self.demandPrice = self.makeArray(data["demandPrice"])
        self.demandCapCost = self.makeArray(data["demandCapCost"])
        
        self.supplyPrice = self.makeArray(data["supplyPrice"])
        self.supplyCapCost = self.makeArray(data["supplyCapCost"])
        self.fuelQuantity = self.makeArray(data["fuelQuantity"])
        
        self.elasticity = self.makeArray(data["elasticity"], reshape=False)
        self.freq = self.makeArray(data["freq"], reshape=False)
        self.fuelUnits = data["fuelUnits"]
        self.priceUnits = data["priceUnits"]
        self.year = data["year"]
        
        self.demLCC = self.zeroArray(numrows=self.numTechs, numcols=0)
        self.ms = self.zeroArray(numrows=self.numTechs, numcols=0)
        
        self.supLCC = self.zeroArray(numrows=self.numTechs, numcols=1)
        self.supMS = self.zeroArray(numrows=self.numTechs, numcols=0)      
        
        self.history = {}
        self.history["demandPrice"] = self.demandPrice
        self.history["demandLCC"] = self.demLCC
        self.history["demandMS"] = self.ms
        self.history["demandQ"] = self.productQuantity
        
        self.history["supplyPrice"] = self.supplyPrice
        self.history["supplyLCC"] = self.supLCC
        self.history["supplyMS"] = self.supMS
        self.history["supplyQ"] = self.fuelQuantity      
        
        

        
    def initialize(self, verbose=True):
        
        # Estimate Quantity Demanded
        # Get LCC from Price and Capital Cost
        self.demLCC = self.getLCC(self.demandPrice, capCost=self.demandCapCost, freq=self.freq)
        self.history["demandLCC"] = self.demLCC
        
        self.demMS = self.getMS(lcc=self.demLCC)
        self.history["demandMS"] = self.demMS
        self.demQuantity = self.getQuantity(self.demMS, self.productQuantity)
        self.history["demandQ"] = self.demQuantity
        
        # Estimate Supply Price
        # Get LCC from Price and Capital Cost
        self.supLCC = self.getLCC(self.supplyPrice, capCost=self.supplyCapCost, freq=self.freq)
        self.history["supplyLCC"] = self.supLCC
        
        self.supMS = self.getMS(lcc=self.supLCC)
        self.history["supplyMS"] = self.supMS 
        
        # get price, send back to demand
        self.demandPrice = self.getPrice(self.supMS, self.supLCC)
        self.history["supplyPrice"] = self.demandPrice
        
        
    def equilibrium(self):
        self.initialize()
        
        
        
        
        
        
        
    def getLCC(self, price, capCost=None ,freq=None, index = -1):
        ngroups, nreps = price.shape
        # ML! might have to deal w corner cases due to None 
        
        lcc = self.zeroArray(numrows=self.numTechs, numcols=1)
        
        if np.any(capCost, None):
            capCost = self.zeroArray(numrows=ngroups, numcols=0)
            
        ccSize = capCost.ndim    
            
        if np.any(freq, None):
            freq = np.ones(ngroups)
        if price.shape[1] == 1:
            index = 0

        for tech in range(ngroups):
            cost = price[tech, index] * freq[tech]
            if ccSize == 1:
                capitalCost = capCost[tech]
            elif ccSize == 2:   
                capitalCost = capCost[tech, index]
            else:
                print("Implementation only supports capital cost arrays of 1 or 2D")
            lcc[tech, 0] = cost + capitalCost
        return lcc    
            
            
    def getMS(self, lcc, index=-1, atol=1e-8):
        ngroups, nreps = lcc.shape
        if nreps == 1:
            index = 0
        full_cost_v = np.sum(np.power(lcc, self.elasticity))
        ms = np.array([np.power(lcc[nt, index], self.elasticity)/full_cost_v for nt in range(ngroups)])  
        assert np.isclose(np.sum(ms), 1.0, atol=atol)
        return ms
    
    
    
    def getQuantity(self, ms, quantity, index=-1):
        ndims = ms.ndim
        if ndims == 1:
            ngroups = ms.shape[0]
        elif ndims == 2:    
            ngroups, nreps = ms.shape
            if nreps == 1:
                index = 0
        qSize = quantity.ndim
        if qSize == 1:    
            quant = np.array([ms[nt] * quantity for nt in range(ngroups)])
        elif qSize > 1:  # ML! might not work for D > 2   
            quant = np.array([ms[nt] * quantity[index] for nt in range(ngroups)])
        return quant    
    
    def getPrice(self, ms, lcc):
        nrows = lcc.shape[0]
        weightedP = [ms[nr] * lcc[nr] for nr in range(nrows)]
        priceArray = self.makeArray(weightedP, numrows=nrows)
        return priceArray
        
        
        
    
    def makeArray(self, field, numrows=2, dtype=np.float64, order='F', reshape=True):    
        if reshape:
            npArray = np.array(field, dtype=dtype).reshape([numrows, 1], order=order)
        else:   
            npArray = np.array(field, dtype=dtype, order=order)
        return npArray
    
        self.demLCC = np.zeros([self.numTechs, 1], dtype=np.float64, order='F')

    def zeroArray(self, numrows=2, numcols=1, dtype=np.float64, order='F'):
        if numcols == 0:
            zeroArr = np.zeros(numrows, dtype=dtype, order=order)
        else:    
            zeroArr = np.zeros([numrows, numcols], dtype=dtype, order=order)
        return zeroArr
            
                        
        

        

In [ ]:
sd = SupplyDemand(data)

In [ ]:
sd.initialize()

In [ ]:
sd.history["demandMS"]

In [ ]:
        if verbose:
            print('Fuel Quantity Per Tech Demanded:\n')
            for nt in range(self.numTechs):
                print('\t- {}: {:.3f} {}'.format(self.techNames[nt], self.quantity[nt], self.fuelUnits))

            print('\nFuel Price Price Calculated on Demand Side:\n')
            for qt in range(self.numTechs):
                print('\t- {}: {}{:.2f}'.format(self.techNames[qt], self.priceUnits, self.demandLCC[qt]))
                  
        

In [ ]:

        if verbose:
            print('\t*****\nFuel Quantity Supplied Calculated:\n')
            for k in range(self.numFuels):
                print('\t- {}: {:.3f} {}'.format(self.fuelNames[k], quant[k], self.fuelUnits))

            print('\nFuel Generation Cost Calculated:\n')
            for kk in range(self.numFuels):
                print('\t- {}: {}{:.2f}'.format(self.fuelNames[kk], self.priceUnits, self.history["supplyLCC"][0][kk]))                

            
    """
    `OPTIMIZATION` METHOD:
    """

    def equilibrium(self, relStepSize=0.1, tol=1e-2, verbose=True):
        
        # start with iteration 1 because index 0 occuppied by initialization // check indices 
        stepnum = 0

        dPrice = self.demandPrice[-1]
        sPrice = self.supplyPrice[-1]   
        
        # calculated twice just for while condition
        priceDiff = [np.abs(dPrice[idx] - sPrice[idx]) for idx in range(self.numTechs)]
        
        while np.any(np.array(priceDiff) > tol):

            if verbose:
                if stepnum == 0:
                    print('\nIteration number:')
                print(stepnum)
                
            # Demand side
            techDemLCC = [] # store all techs in a list, to append to full nested list for LCC at each iter
            
            # New price diff every iteration
            dPrice = self.demandPrice[-1]
            sPrice = self.supplyPrice[-1]   
            
            dPriceNew = []
            sPriceNew = []
        
            # calculated twice just for while condition
            # ML! IMPORTANT: WILL NEED TO SWITCH TO DICT TO PUT LABELS WHEN FUEL/TECH AREN'T 1-TO-1
            priceDiff = [np.abs(dPrice[idx] - sPrice[idx]) for idx in range(self.numTechs)]
            
            for tech in range(self.numTechs):                
                supplyHigher = sPrice[tech] > dPrice[tech]
                            
                if supplyHigher:
                    dPriceN = dPrice[tech] + relStepSize*priceDiff[tech]
                    sPriceN = sPrice[tech] - relStepSize*priceDiff[tech]
                else:
                    dPriceN = dPrice[tech] - relStepSize*priceDiff[tech]
                    sPriceN = sPrice[tech] + relStepSize*priceDiff[tech]   
                                        
                # TO DO: store dPrice and sPrice in history to view incremental changes
                
                # list of list: index 0 is initialisation (iter 0), sublist is techs
                freq = self.freq[tech]
                capCost = self.demandCapCost[-1][tech]
                techDemLCC.append(dPriceN*freq + capCost)

            # ML!: change dPrice lists to match
            self.demandPrice.append(dPrice)
            
            self.history["demandPrice"][0].append(dPrice)
            self.history["demandLCC"].append(techDemLCC)
            
            full_cost = np.sum(self.history["demandLCC"][-1])
            full_cost_v = np.sum(np.power(self.history["demandLCC"][-1], self.elasticity))
            ms = [np.power(self.history["demandLCC"][-1][nt], self.elasticity)/full_cost_v for nt in range(self.numTechs)]
            self.history["demandMS"].append(ms)    
            
            assert np.isclose(np.sum(ms), 1.0, atol=1e-8)
            
            quant = [ms[j]*self.fullQuantity for j in range(self.numTechs)]
            self.history["demandQ"].append(quant)
            
            if verbose and stepnum % 10 == 0:
                print('Fuel Quantity Demanded Calculated:\n')
                for k in range(self.numTechs):
                    print('\t- {}: {:.3f} {}'.format(self.techNames[k], quant[k], self.fuelUnits))
    
                print('\nTech Operation Price Calculated:\n')
                for kk in range(self.numTechs):
                    print('\t- {}: {}{:.2f}'.format(self.techNames[kk], self.priceUnits, self.history["demandLCC"][-1][kk]))
                    
                    
            # SUPPLY SIDE         
            fuelSupLCC = []
            
            for fuel in range(self.numFuels):
                capCost = self.supplyCapCost[-1][fuel]
                fuelSupLCC.append(sPrice[fuel] + capCost) # remember to arrange indices when fuel and tech aren't 1-to-1
            
            self.supplyPrice.append(sPrice)
            self.history["supplyLCC"].append(fuelSupLCC)    

            full_cost = np.sum(self.history["supplyLCC"][-1])
            full_cost_v = np.sum(np.power(self.history["supplyLCC"][-1], self.elasticity))
            ms = [np.power(self.history["supplyLCC"][-1][i], self.elasticity)/full_cost_v for i in range(self.numFuels)]
            self.history["supplyMS"].append(ms)    
            
            assert np.isclose(np.sum(ms), 1.0, atol=1e-8)
            
            quant = [ms[j]*self.fullQuantity for j in range(self.numTechs)]
            self.history["supplyQ"].append(quant)
            
            if verbose and stepnum % 10 == 0:
                print('\t*****\nFuel Quantity Supplied Calculated:\n')
                for k in range(self.numFuels):
                    print('\t- {}: {:.3f} {}'.format(self.fuelNames[k], quant[k], self.fuelUnits))
    
                print('\nFuel Generation Cost Calculated:\n')
                for kk in range(self.numFuels):
                    print('\t- {}: {}{:.2f}'.format(self.fuelNames[kk], self.priceUnits, self.history["supplyLCC"][stepnum][kk]))    
                    
            stepnum += 1    
            
        if verbose:
            print('Final Fuel Quantity Demanded Calculated:\n')
            for k in range(self.numTechs):
                print('\t- {}: {:.3f} {}'.format(self.techNames[k], self.history["demandQ"][-1][k], self.fuelUnits))
    
            print('\nFinal Tech Operation Price Calculated:\n')
            for kk in range(self.numTechs):
                print('\t- {}: {}{:.2f}'.format(self.techNames[kk], self.priceUnits, self.history["demandLCC"][-1][kk]))
                            
            print('\t*****\nFinal Fuel Quantity Supplied Calculated:\n')
            for k in range(self.numFuels):
                print('\t- {}: {:.3f} {}'.format(self.fuelNames[k], self.history["supplyQ"][-1][kk], self.fuelUnits))
    
            print('\nFinal Fuel Generation Cost Calculated:\n')
            for kk in range(self.numFuels):
                print('\t- {}: {}{:.2f}'.format(self.fuelNames[kk], self.priceUnits, self.history["supplyLCC"][stepnum][kk]))              
            
                


In [ ]:
mod = SupplyDemand(data)

In [ ]:
mod.initialize(verbose=False)

mod.equilibrium(verbose=True)

In [ ]:
qSupElec = [mod.history['supplyQ'][num][0] for num in range(len(mod.history['supplyQ']))]
pSupElec = [mod.history['supplyLCC'][num][0] for num in range(len(mod.history['supplyLCC']))]

qDemElec = [mod.history['demandQ'][num][0] for num in range(len(mod.history['demandQ']))]
pDemElec = [mod.history['demandLCC'][num][0] for num in range(len(mod.history['demandLCC']))]

In [ ]:
plt.plot(qSupElec, pSupElec)
plt.plot(qDemElec, pDemElec)
plt.show()

In [ ]:
mod.history['demandPrice']

In [ ]:
dPriceElec = np.linspace(40,(40+15)/2, 100)
sPriceElec = np.linspace(15,(40+15)/2, 100)
ElecQuant = np.linspace(61.538,  )

dPriceGas = np.linspace(40,(40+15)/2, 100)
sPriceGas = np.linspace(15,(40+15)/2, 100)

In [ ]:
priceGas

In [ ]:
mod.equilibrium()

In [ ]:
np.linspace()

In [ ]:
colour = ['moccasin', 'lightblue']
plt.pie([elecHeatMarketShare_t0, gasHeatMarketShare_t0],
        labels=['electric heater', 'gas heater'],
        colors=colour,
        autopct='%.2f');
plt.suptitle('Heater Market Share')
plt.axis('equal')
plt.show()

## Supply 

In [ ]:
# electricity supply
elecPrice_t1 = elecPrice_t0
elecLCC_t1 = elecPrice_t1 + elecCapCost

# gas supply
gasPrice_t1 = gasPrice_t0
gasLCC_t1 = gasPrice_t1 + gasCapCost

supply_cost_t1 = elecLCC_t1 + gasLCC_t1 

elecMarketShare_t1 = (elecLCC_t1/supply_cost_t1)**elasticity
gasMarketShare_t1 = (gasLCC_t1/supply_cost_t1)**elasticity

assert np.isclose(gasMarketShare_t1 + elecMarketShare_t1, 1.0, atol=1e-8)

elecQuant_t1 = elecMarketShare_t1*fullQuantity
gasQuant_t1 = gasMarketShare_t1*fullQuantity

print('Fuel Quantity Supplied Calculated:\n')
print('\t- Electricity: {:.3f} MJ\n\t- Gas: {:.3f} MJ\n'.format(elecQuant_t1, gasQuant_t1))

print('Fuel Production Cost Calculated:\n')
print('\t- Electricity: ${:.2f}\n\t- Gas: ${:.2f}\n'.format(elecLCC_t1, gasLCC_t1))

In [ ]:
colour = ['lightgreen', 'thistle']
plt.pie([elecMarketShare_t1, gasMarketShare_t1],
        labels=['electricity', 'gas'],
        colors=colour,
        autopct='%.2f');
plt.suptitle('Fuel Production Market Share')
plt.axis('equal')
plt.show()

### Iterate

In [ ]:
elecHeatLCC_t1 = (elecLCC_t1 + elecHeatLCC_t0)/2
gasHeatLCC_t1 = (gasLCC_t1 + gasHeatLCC_t0)/2

In [ ]:
full_cost_t1 = elecHeatLCC_t1 + gasHeatLCC_t1 

elecHeatMarketShare_t1 = (elecHeatLCC_t1/full_cost_t1)**elasticity
gasHeatMarketShare_t1 = (gasHeatLCC_t1/full_cost_t1)**elasticity

assert np.isclose(gasHeatMarketShare_t1 + elecHeatMarketShare_t1, 1.0, atol=1e-8)

elecQuant_t1 = elecHeatMarketShare_t1*fullQuantity
gasQuant_t1 = gasHeatMarketShare_t1*fullQuantity

print('Fuel Quantity Demanded Calculated:\n')
print('\t- Electricity: {:.3f} MJ\n\t- Gas: {:.3f} MJ\n'.format(elecQuant_t1, gasQuant_t1))

print('Tech Operation Price Calculated:\n')
print('\t- Electric Heater: ${:.2f}\n\t- Gas Heater: ${:.2f}\n'.format(elecHeatLCC_t1, gasHeatLCC_t1))